In [3]:
import pandas as pd
import sqlite3
import os 

In [4]:
cxn = sqlite3.connect('AdventureWorks.db')

In [5]:
#Exercise 1
rating_ranking = '''select p.productid productid, p.Name NAME, avg(r.rating) avgrating, 
                    (case r.rating when 5 then 1
                            when 4 then 2
                            when 3 then 3
                            when 2 then 4
                            when 1 then 5
                    end) num_ratings 
                    from product p join productreview r 
                    on p.productid = r.productid group by p.productid order by num_ratings'''
pd.read_sql(rating_ranking, cxn)

,productid,NAME,avgrating,num_ratings
0,709,"Mountain Bike Socks, M",5.0,1
1,798,"Road-550-W Yellow, 40",5.0,1
2,937,HL Mountain Pedal,3.0,4


In [6]:
# Name your variable productmodelid_description
# YOUR CODE HERE
#Exercise 2.1
productmodelid_description = '''select p.productmodelid, d.description 
                from productmodelproductdescriptionculture p, 
                productdescription d where p.productdescriptionid = d.productdescriptionid
                and p.cultureid = 'en' '''
pd.read_sql(productmodelid_description, cxn)
#cxn.execute(productmodelid_description).fetchall()

,productmodelid,description
0,1,"Light-weight, wind-resistant, packs to fit int..."
1,2,Traditional style with a flip-up brim; one-siz...
2,3,"Synthetic palm, flexible knuckles, breathable ..."
3,4,"Full padding, improved finger flex, durable pa..."
4,5,Each frame is hand-crafted in our Bothell faci...
...,...,...
122,123,Replacement mountain wheel for entry-level rider.
123,124,Replacement mountain wheel for the casual to s...
124,125,High-performance mountain replacement wheel.
125,126,Replacement road rear wheel for entry-level cy...


In [8]:
# Name your variable description_totalorders
# YOUR CODE HERE
#Exercise 2.2
description_totalorders='''select v.*, p.Name, sum(s.orderqty) total_orders from product_view v, product p, 
                salesorderdetail s where v.productmodelid = p.productmodelid 
                and p.productid = s.productid group by p.productid order by total_orders desc limit 10'''
pd.read_sql(description_totalorders, cxn)

,productmodelid,description,NAME,total_orders
0,2,Traditional style with a flip-up brim; one-siz...,AWC Logo Cap,8311
1,111,AWC logo water bottle - holds 30 oz; leak-proof.,Water Bottle - 30 oz.,6815
2,33,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Blue",6743
3,11,Unisex long-sleeve AWC logo microfiber cycling...,"Long-Sleeve Logo Jersey, L",6592
4,33,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Black",6532
5,33,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",6266
6,1,"Light-weight, wind-resistant, packs to fit int...","Classic Vest, S",4247
7,114,"Includes 8 different size patches, glue and sa...",Patch Kit/8 Patches,3865
8,32,Short sleeve classic breathable jersey with su...,"Short-Sleeve Classic Jersey, XL",3864
9,11,Unisex long-sleeve AWC logo microfiber cycling...,"Long-Sleeve Logo Jersey, M",3636


In [9]:
#Exercise 3.1
quantities_ordered = '''select productid, sum(orderqty) quantity
                    from salesorderdetail group by productid order by productid'''
pd.read_sql(quantities_ordered, cxn)

,productid,quantity
0,707,6266
1,708,6532
2,709,1107
3,710,90
4,711,6743
...,...,...
261,994,378
262,996,543
263,997,656
264,998,1556


In [10]:
#Exercise 3.2
products_prices = '''select p.productid,  c.name category, sub.name subcategory, p.listprice 
                    from product p, productcategory c, productsubcategory sub
                    where c.productcategoryid = sub.productcategoryid and 
                    sub.productsubcategoryid = p.productsubcategoryid  
                    group by p.productid'''
pd.read_sql(products_prices, cxn)

,productid,category,subcategory,listprice
0,680,Components,Road Frames,1431.50
1,706,Components,Road Frames,1431.50
2,707,Accessories,Helmets,34.99
3,708,Accessories,Helmets,34.99
4,709,Clothing,Socks,9.50
...,...,...,...,...
290,995,Components,Bottom Brackets,101.24
291,996,Components,Bottom Brackets,121.49
292,997,Bikes,Road Bikes,539.99
293,998,Bikes,Road Bikes,539.99


In [12]:
#Exercise 3.3
prices_quantities = '''with quantities_ordered as 
                        (''' + quantities_ordered + '''
                        ), product_prices as 
                        (''' + products_prices + ''')
                        select category, subcategory, avg(product_prices.listprice) average_price_in_subcategory, 
                    sum(quantity) total_items_sold_in_subcategory from quantities_ordered, product_prices where 
                    quantities_ordered.productid = product_prices.productid
                    group by category, subcategory '''
pd.read_sql(prices_quantities, cxn)

,category,subcategory,average_price_in_subcategory,total_items_sold_in_subcategory
0,Accessories,Bike Racks,120.000000,3166
1,Accessories,Bike Stands,159.000000,249
2,Accessories,Bottles and Cages,7.990000,10552
3,Accessories,Cleaners,7.950000,3319
4,Accessories,Fenders,21.980000,2121
5,Accessories,Helmets,34.990000,19541
6,Accessories,Hydration Packs,54.990000,2761
7,Accessories,Locks,25.000000,1087
8,Accessories,Pumps,19.990000,1130
9,Accessories,Tires and Tubes,19.482727,18006


In [36]:
#Exercise 3.3
prices_quantities = '''with quantities_ordered as 
                    (
                      select productid prodid, sum(orderqty) quantity
                    from salesorderdetail group by productid
                    ), product_prices as 
                    (
                      select p.productid,  c.name category, sub.name subcategory, 
                      p.listprice 
                    from product p, productcategory c, productsubcategory sub
                    where c.productcategoryid = sub.productcategoryid and 
                    sub.productsubcategoryid = p.productsubcategoryid  
                    group by p.productid
                    )
                    select category, subcategory, sum(product_prices.listprice)/count(product_prices.listprice) average_price_in_subcategory, 
                    sum(quantity) total_items_sold_in_subcategory from quantities_ordered, product_prices where 
                    prodid = product_prices.productid
                    group by category, subcategory'''
pd.read_sql(prices_quantities, cxn)

,category,subcategory,average_price_in_subcategory,total_items_sold_in_subcategory
0,Accessories,Bike Racks,120.000000,3166
1,Accessories,Bike Stands,159.000000,249
2,Accessories,Bottles and Cages,7.990000,10552
3,Accessories,Cleaners,7.950000,3319
4,Accessories,Fenders,21.980000,2121
5,Accessories,Helmets,34.990000,19541
6,Accessories,Hydration Packs,54.990000,2761
7,Accessories,Locks,25.000000,1087
8,Accessories,Pumps,19.990000,1130
9,Accessories,Tires and Tubes,19.482727,18006


In [61]:
#Exercise 4
salesperson_sales='''select businessentityid, format("%2.4f",salesytd) salesytd from 
                    salesperson order by salesytd desc limit 5'''
pd.read_sql(salesperson_sales, cxn)

,businessentityid,salesytd
0,274,559697.5639
1,287,519905.9320
2,276,4251368.5497
3,289,4116871.2277
4,275,3763178.1787


In [15]:
#Exercise 5
salesperson_totalsales = '''select salespersonid, trim(format("%2.10f", sum(subtotal)), '0') totalsales 
                      from salesorderheader where orderdate >= '2014-01-01' and salespersonid <>'' 
                      group by salespersonid order by sum(subtotal) desc limit 5'''
pd.read_sql(salesperson_totalsales, cxn)

,salespersonid,totalsales
0,289,1382996.5839
1,276,1271088.5216
2,275,1057247.378599999
3,282,1044810.8277
4,277,1040093.4071


In [16]:
#Exercise 6.1
order_ordertotal = '''select d.salesorderid, 
                    sum((d.unitprice - 0.01*(d.unitpricediscount))* d.orderqty) as ordertotal 
                    from salesorderdetail d, salesorderheader h where  d.salesorderid = h.salesorderid 
                    group by d.salesorderid'''
pd.read_sql(order_ordertotal, cxn)

,salesorderid,ordertotal
0,43659,20565.6206
1,43660,1294.2529
2,43661,32726.4786
3,43662,28832.5289
4,43663,419.4589
...,...,...
31460,75119,42.2800
31461,75120,84.9600
31462,75121,74.9800
31463,75122,30.9700


In [38]:
#Exercise 6.2
salesperson_ordertotal= "with sales2014 as (" + order_ordertotal + ")" + '''
                        select h.salespersonid, format("%2.10f",sum(o.ordertotal)) ordertotalsum from sales2014 o, 
                        salesorderheader h where h.salesorderid = o.salesorderid 
                        and h.orderdate>='2014-01-01' and h.orderdate<='2014-12-31' and h.salespersonid <>''
                        group by h.salespersonid order by sum(o.ordertotal) desc limit 5'''
pd.read_sql(salesperson_ordertotal,cxn)

,salespersonid,ordertotalsum
0,289,1387765.1179000000
1,276,1281333.9855000000
2,275,1060241.4868999990
3,282,1047765.9062000000
4,277,1042437.9607000000


In [44]:
# Exercise 7
salesperson_ordertotal= "with sales2014 as (" + order_ordertotal + ")" + '''
                        select h.salespersonid, format("%2.10f",sum(o.ordertotal)) ordertotalsum from sales2014 o, 
                        salesorderheader h where h.salesorderid = o.salesorderid 
                        and h.orderdate>='2014-01-01' and h.orderdate<='2014-12-31' and h.salespersonid <>''
                        group by h.salespersonid order by ordertotalsum'''
salesperson_ordertotal_commission = "with salesperson_commission as (" + salesperson_ordertotal + ''') 
                                    select salespersonid, format("%2.11f",ordertotalsum) ordertotalsum, s.commissionpct from salesperson_commission, 
                                    salesperson s where salespersonid = s.businessentityid'''
pd.read_sql(salesperson_ordertotal_commission, cxn)

,salespersonid,ordertotalsum,commissionpct
0,274,179335.71119999990,0.000
1,275,1060241.48689999800,0.012
2,276,1281333.98549999900,0.015
3,277,1042437.96070000000,0.015
4,278,436670.85689999990,0.010
5,279,788115.45379999990,0.010
6,280,507616.53130000020,0.010
7,281,778925.39840000010,0.010
8,282,1047765.90619999900,0.015
9,283,492038.43520000010,0.012


In [19]:
#Exercise 8.1
salesperson_currency_id = '''select h.salespersonid salespersonid, h.salesorderid salesorderid, 
                            case when h.currencyrateid = '' then 'None' 
                                else h.currencyrateid
                            end currencyrateid,
                            case when h.currencyrateid <> '' then 
                                (select currencyrate.tocurrencycode from currencyrate, salesorderheader 
                                where salesorderheader.currencyrateid = currencyrate.currencyrateid 
                                and salesorderid = salesorderheader.salesorderid) 
                            else 'None'
                            end tocurrencycode
                            from salesperson s, salesorderheader h, currencyrate c where 
                            h.salespersonid = s.businessentityid 
                            and h.orderdate>='2014-01-01' and h.orderdate<='2014-12-31' group by salesorderid order by 
                            h.salespersonid limit 10'''
pd.read_sql(salesperson_currency_id, cxn)

,salespersonid,salesorderid,currencyrateid,tocurrencycode
0,274,65294,None,None
1,274,65298,None,None
2,274,67277,None,None
3,274,67286,11427,CAD
4,274,69528,None,None
5,274,69545,None,None
6,274,71779,None,None
7,274,71919,None,None
8,275,65205,None,None
9,275,65215,None,None


In [47]:
#Exercise 8.2
salesperson_currency_code = '''select s.salespersonid salespersonid, s.salesorderid salesorderid, 
                            (case s.currencyrateid  
                                when '' then 'USA' 
                                else s.currencyrateid 
                            end) currencyrateid,
                            (case s.currencyrateid
                                when '' then 'USA'
                                else c.tocurrencycode
                            end) tocurrencycode
                            from salesorderheader s left join CurrencyRate c on 
                            s.currencyrateid = c.currencyrateid
                            where s.orderdate >= '2014-01-01' and s.orderdate <= '2014-12-31' 
                            group by s.salesorderid order by s.salespersonid,
                            s.salesorderid limit 10'''
pd.read_sql(salesperson_currency_code, cxn)

,salespersonid,salesorderid,currencyrateid,tocurrencycode
0,274,65294,USA,USA
1,274,65298,USA,USA
2,274,67277,USA,USA
3,274,67286,11427,CAD
4,274,69528,USA,USA
5,274,69545,USA,USA
6,274,71779,USA,USA
7,274,71919,USA,USA
8,275,65205,USA,USA
9,275,65215,USA,USA


In [20]:
#Exercise 8.2
salesperson_currency_code = '''select h.salespersonid salespersonid, h.salesorderid salesorderid, 
                (case h.currencyrateid when '' then 'USD' 
                    else h.currencyrateid 
                end) currencyrateid, 
                (case h.currencyrateid when '' then 'USD' 
                    else (select c.tocurrencycode from currencyrate c where h.currencyrateid = c.currencyrateid)
                end) tocurrencycode from salesorderheader h, salesperson s where h.salespersonid = s.businessentityid 
                and h.orderdate>='2014-01-01' and h.orderdate<='2014-12-31' 
                group by salesorderid order by salespersonid limit 10'''
pd.read_sql(salesperson_currency_code, cxn)

,salespersonid,salesorderid,currencyrateid,tocurrencycode
0,274,65294,USD,USD
1,274,65298,USD,USD
2,274,67277,USD,USD
3,274,67286,11427,CAD
4,274,69528,USD,USD
5,274,69545,USD,USD
6,274,71779,USD,USD
7,274,71919,USD,USD
8,275,65205,USD,USD
9,275,65215,USD,USD


In [ ]:
#Exercise 8.2
salesperson_currency_code = '''select s.salespersonid salespersonid, s.salesorderid salesorderid, 
                            (case s.currencyrateid  
                                when '' then 'USA' 
                                else s.currencyrateid 
                            end) currencyrateid,
                            (case s.currencyrateid
                                when '' then 'USA'
                                else c.tocurrencycode
                            end) tocurrencycode
                            from salesorderheader s left join CurrencyRate c on 
                            s.currencyrateid = c.currencyrateid
                            where s.orderdate >= '2014-01-01' and s.orderdate <= '2014-12-31' 
                            group by s.salesorderid order by s.salespersonid,
                            s.salesorderid limit 10'''
pd.read_sql(salesperson_currency_code, cxn)

In [59]:
#Exercise 9
salesperson_ordertotal= "with sales2014 as (" + order_ordertotal + ")" + '''
                        select h.salespersonid, format("%2.10f",sum(o.ordertotal)) ordertotalsum from sales2014 o, 
                        salesorderheader h where h.salesorderid = o.salesorderid 
                        and h.orderdate>='2014-01-01' and h.orderdate<='2014-12-31' and h.salespersonid <>''
                        group by h.salespersonid order by ordertotalsum'''
salesperson_ordertotal_commission = "with salesperson_commission as (" + salesperson_ordertotal + ''') 
                                    select salespersonid, format("%2.11f",ordertotalsum) ordertotalsum, s.commissionpct from salesperson_commission, 
                                    salesperson s where salespersonid = s.businessentityid'''
                                    
salesperson_ranking_currency = '''with commission as 
                                (''' + salesperson_ordertotal_commission +  ''')   
                               select s.salespersonid salespersonid, 
                            (case s.currencyrateid  
                                when '' then 'USA' 
                                else c.fromcurrencycode
                            end) currencyrateid,
                            (case s.currencyrateid
                                when '' then 'USA'
                                else c.tocurrencycode
                            end) tocurrencycode
                            from salesorderheader s left join CurrencyRate c on 
                            s.currencyrateid = c.currencyrateid where s.salespersonid <>'' group by salespersonid'''
pd.read_sql(salesperson_ranking_currency, cxn)

,salespersonid,currencyrateid,tocurrencycode
0,274,USA,USA
1,275,USA,USA
2,276,USA,USA
3,277,USA,USA
4,278,USD,CAD
5,279,USA,USA
6,280,USA,USA
7,281,USA,USA
8,282,USD,CAD
9,283,USA,USA


In [56]:
pd.read_sql("select * from CurrencyRate", cxn)

,currencyrateid,currencyratedate,fromcurrencycode,tocurrencycode,averagerate,endofdayrate,modifieddate
0,1,2011-05-31 00:00:00,USD,ARS,1.0000,1.0002,2011-05-31 00:00:00
1,2,2011-05-31 00:00:00,USD,AUD,1.5491,1.5500,2011-05-31 00:00:00
2,3,2011-05-31 00:00:00,USD,BRL,1.9379,1.9419,2011-05-31 00:00:00
3,4,2011-05-31 00:00:00,USD,CAD,1.4641,1.4683,2011-05-31 00:00:00
4,5,2011-05-31 00:00:00,USD,CNY,8.2781,8.2784,2011-05-31 00:00:00
...,...,...,...,...,...,...,...
13527,13528,2014-05-27 00:00:00,USD,USD,1.0000,1.0000,2014-01-02 17:39:45.26
13528,13529,2014-05-28 00:00:00,USD,USD,1.0000,1.0000,2014-01-02 17:39:45.26
13529,13530,2014-05-29 00:00:00,USD,USD,1.0000,1.0000,2014-01-02 17:39:45.26
13530,13531,2014-05-30 00:00:00,USD,USD,1.0000,1.0000,2014-01-02 17:39:45.26


In [21]:
cxn.close()

In [77]:
# Ex. 1
assert "rating_ranking" in globals(), "Ex. 1 - Remember that your variable's name should be `rating_ranking`!"
rating_ranking_result = pd.read_sql(rating_ranking, cxn)
assert len(rating_ranking_result) > 0, "Ex. 1 - Your code is not producing any output! (ie., a table with lenght zero)"
assert set(rating_ranking_result.columns) == {'NAME', 'avgrating', 'num_ratings', 'productid'}, "Ex. 1 - Your query result doesn't have exactly these columns: 'NAME', 'avgrating', 'num_ratings', 'productid'"
print("Exercise 1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [78]:
# Ex. 2.1
assert "productmodelid_description" in globals(), "Ex. 2.1 - Remember that your variable's name should be `productmodelid_description`!"
productmodelid_description_result = pd.read_sql(productmodelid_description, cxn)
assert len(productmodelid_description_result) == 127, "Ex. 2.1 - There are 127 product models in the database, but your query produces a different number. Make sure that you don't have any LIMIT clauses in this exercise!"
assert set(productmodelid_description_result.columns) == {'description', 'productmodelid'}, "Ex. 2.1 - Your query result doesn't have exactly these columns: 'description', 'productmodelid'"
print("Exercise 2.1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 2.1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [83]:
#Ex. 2.2
assert "description_totalorders" in globals(), "Ex. 2.2 - Remember that your variable's name should be `description_totalorders`!"
description_totalorders_result = pd.read_sql(description_totalorders, cxn)
assert len(description_totalorders_result) == 10, "Ex. 2.2 - Remember to use LIMIT 10 in your query! This is a top 10!"
assert set(description_totalorders_result.columns) == {'NAME', 'description', 'productmodelid', 'total_orders'}, "Ex. 2.2 - Your query result doesn't have exactly these columns: 'NAME', 'description', 'productmodelid', 'total_orders'"
print("Exercise 2.2 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 2.2 looks fine for now. You will get your final grade after we've reviewed your submission.


In [84]:
# Ex. 3.1
assert "quantities_ordered" in globals(), "Ex. 3.1 - Remember that your variable's name should be `quantities_ordered`!"
quantities_ordered_result = pd.read_sql(quantities_ordered, con=cxn)
assert len(quantities_ordered_result) == 266, "Ex. 2 - There are 266 products in the database that have associated quantities, but your query produces a different number. Make sure that you don't have any LIMIT clauses in this exercise and don't filter by culture!"
assert set(quantities_ordered_result.columns) == {'productid', 'quantity'}, "Ex. 3.1 - Your query result doesn't have exactly these columns: 'productid', 'quantity'"
print("Exercise 3.1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 3.1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [85]:
# Ex. 3.2
assert "products_prices" in globals(), "Ex. 3.2 - Remember that your variable's name should be `products_prices`!"
products_prices_result = pd.read_sql(products_prices, cxn)
assert len(products_prices_result) == 295, "Ex. 3.2 - There are 295 products in the database that have prices, but your query produces a different number. Make sure that you don't have any LIMIT clauses in this exercise!"
assert set(products_prices_result.columns) == {'category', 'listprice', 'productid', 'subcategory'}, "Ex. 3.2 - Your query result doesn't have exactly these columns: 'category', 'listprice', 'productid', 'subcategory'"
print("Exercise 3.2 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 3.2 looks fine for now. You will get your final grade after we've reviewed your submission.


In [37]:
# Ex. 3.3
assert "prices_quantities" in globals(), "Ex. 3.3 - Remember that your variable's name should be `prices_quantities`!"
prices_quantities_result = pd.read_sql(prices_quantities, cxn)
assert len(prices_quantities_result) == 35, "Ex. 3.3 - There are 35 subcategories in the database, but your query produces a different number. Make sure that you don't have any LIMIT clauses in this exercise!"
assert set(prices_quantities_result.columns) == {'average_price_in_subcategory', 'category', 'subcategory', 'total_items_sold_in_subcategory'}, "Ex. 3.3 - Your query result doesn't have exactly these columns: 'average_price_in_subcategory', 'category', 'subcategory', 'total_items_sold_in_subcategory'"
print("Exercise 3.3 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 3.3 looks fine for now. You will get your final grade after we've reviewed your submission.


In [22]:
# Ex. 4
assert "salesperson_sales" in globals(), "Ex. 4 - Remember that your variable's name should be `salesperson_sales`!"
salesperson_sales_result = pd.read_sql(salesperson_sales, cxn)
assert len(salesperson_sales_result) == 5, "Ex. 4 - This is a top 5. Remember to use LIMIT!"
assert set(salesperson_sales_result.columns) == {'businessentityid', 'salesytd'}, "Ex. 4 - Your query result doesn't have exactly these columns: 'businessentityid', 'salesytd'"
print("Exercise 4 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 4 looks fine for now. You will get your final grade after we've reviewed your submission.


In [23]:
# Ex. 5
assert "salesperson_totalsales" in globals(), "Ex. 5 - Remember that your variable's name should be `salesperson_totalsales`!"
salesperson_totalsales_result = pd.read_sql(salesperson_totalsales, cxn)
assert len(salesperson_totalsales_result) == 5, "Ex. 5 - This is a top 5. Remember to use LIMIT!"
assert set(salesperson_totalsales_result.columns) == {'salespersonid', 'totalsales'}, "Ex. 5 - Your query result doesn't have exactly these columns: 'salespersonid', 'totalsales'"
print("Exercise 5 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 5 looks fine for now. You will get your final grade after we've reviewed your submission.


In [24]:
# Ex. 6.1
assert "order_ordertotal" in globals(), "Ex. 6.1 - Remember that your variable's name should be `order_ordertotal`!"
order_ordertotal_result = pd.read_sql(order_ordertotal, cxn)
assert len(order_ordertotal_result) == 31465, "Ex. 6.1 - There are more than 31,000 orders in the database. Remember to NOT use LIMIT here!"
assert set(order_ordertotal_result.columns) == {'ordertotal', 'salesorderid'}, "Ex. 6.1 - Your query result doesn't have exactly these columns: 'ordertotal', 'salesorderid'"
print("Exercise 6.1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 6.1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [39]:
# Ex. 6.2
assert "salesperson_ordertotal" in globals(), "Ex. 6.2 - Remember that your variable's name should be `salesperson_ordertotal`!"
salesperson_ordertotal_result = pd.read_sql(salesperson_ordertotal, cxn)
assert len(salesperson_ordertotal_result) == 5, "Ex. 6.2 - There are too many or too few rows in your result. Remember to use LIMIT here!"
assert set(salesperson_ordertotal_result.columns) == {'ordertotalsum', 'salespersonid'}, "Ex. 6.2 - Your query result doesn't have exactly these columns: 'ordertotalsum', 'salespersonid'"
print("Exercise 6.2 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 6.2 looks fine for now. You will get your final grade after we've reviewed your submission.


In [40]:
# Ex. 7
assert "salesperson_ordertotal_commission" in globals(), "Ex. 7 - Remember that your variable's name should be `salesperson_ordertotal_commission`!"
salesperson_ordertotal_commission_result = pd.read_sql(salesperson_ordertotal_commission, cxn)
assert len(salesperson_ordertotal_commission_result) == 17, "Ex. 7 - There are too many or too few rows in your result. Remember to NOT use LIMIT here!"
assert set(salesperson_ordertotal_commission_result.columns) == {'commissionpct', 'ordertotalsum', 'salespersonid'}, "Ex. 7 - Your query result doesn't have exactly these columns: 'commissionpct', 'ordertotalsum', 'salespersonid'"
print("Exercise 7 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 7 looks fine for now. You will get your final grade after we've reviewed your submission.


In [41]:
 #Ex. 8.1
assert "salesperson_currency_id" in globals(), "Ex. 8.1 - Remember that your variable's name should be `salesperson_currency_id`!"
salesperson_currency_result = pd.read_sql(salesperson_currency_id, cxn)
assert len(salesperson_currency_result) == 10, "Ex. 8.1 - There are too many or too few rows in your result. Remember to use LIMIT here!"
assert set(salesperson_currency_result.columns) == {'salespersonid', 'salesorderid', 'currencyrateid','tocurrencycode'}, "Ex. 8.1 - Your query result doesn't have exactly these columns: 'salespersonid', 'salesorderid', 'currencyrateid','tocurrencycode'"
print("Exercise 8.1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 8.1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [48]:
# Ex. 8.2
assert "salesperson_currency_code" in globals(), "Ex. 8.2 - Remember that your variable's name should be `salesperson_currency_code`!"
salesperson_currency_result2 = pd.read_sql(salesperson_currency_code, cxn)
assert len(salesperson_currency_result2) == 10, "Ex. 8.2 - There are too many or too few rows in your result. Remember to use LIMIT here!"
assert set(salesperson_currency_result2.columns) == {'salespersonid', 'salesorderid','currencyrateid','tocurrencycode'}, "Ex. 8.2 - Your query result doesn't have exactly these columns: 'salespersonid', 'salesorderid', 'currencyrateid','tocurrencycode'"
print("Exercise 8.2 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 8.2 looks fine for now. You will get your final grade after we've reviewed your submission.


In [60]:
# Ex. 9
assert "salesperson_ranking_currency" in globals(), "Ex. 9 - Remember that your variable's name should be `salesperson_ranking_currency`!"
salesperson_ranking_currency_result = pd.read_sql(salesperson_ranking_currency, cxn)
assert len(salesperson_ranking_currency_result) == 21, "Ex. 9 - There are too many or too few rows in your result. Remember to NOT use LIMIT here!"
assert set(salesperson_ranking_currency_result.columns) == {'salespersonid', 'tocurrencycode', 'ordertotalsum', 'commissionpct'}, "Ex. 9 - Your query result doesn't have exactly these columns: 'salespersonid', 'tocurrencycode', 'ordertotalsum', 'commissionpct'"
print("Exercise 9 looks fine for now. You will get your final grade after we've reviewed your submission.")

AssertionError: Ex. 9 - There are too many or too few rows in your result. Remember to NOT use LIMIT here!